## 5: Composing holoviews plots

So far we have generated plots using hvplot, but we haven't discussed all the things that you can do with these outputs.

### Read in the data

In [ ]:
import dask
import dask.dataframe as dd

In [ ]:
df = dd.read_parquet('../data/earthquakes.parq')
df.head()

In [ ]:
cleaned_df = df.copy()
cleaned_df['mag'] = df.mag.where(df.mag > 0)
cleaned_reindexed_df = cleaned_df.set_index(cleaned_df.time)

If you have enough RAM:

In [ ]:
cleaned_reindexed_df = cleaned_reindexed_df.persist()

### Resampling data

In [ ]:
import hvplot.dask

In [ ]:
monthly_count = cleaned_reindexed_df.id.resample('1M').count()
monthly_count_plot = monthly_count.hvplot(title='Monthly count')
monthly_count_plot

In [ ]:
print(monthly_count_plot)

In [ ]:
monthly_mean_magnitude = persisted_cleaned_reindexed_df.mag.resample('1M').mean()
monthly_mean_magnitude_plot = monthly_mean_magnitude.hvplot(title='Monthly mean magnitude')
monthly_mean_magnitude_plot

In [ ]:
print(monthly_mean_magnitude_plot)

In [ ]:
(monthly_mean_magnitude_plot + monthly_count_plot).cols(1)

## Adding a third dimension

So far we have used location, time, magnitude, and depth. Now let's filter the earthquakes to only include the really high intensity ones. We can add extra dimensions to the visualization by using color in addition to x and y.

In [ ]:
most_severe = persisted[persisted.mag >= 7].compute()

In [ ]:
most_severe.plot.scatter(x='longitude', y='latitude', c='mag')

In [ ]:
most_severe.hvplot.scatter(x='longitude', y='latitude', c='mag')

Tweaking the options to create a better plot

In [ ]:
most_severe.hvplot.scatter(x='longitude', y='latitude', c='mag', hover_cols=['place', 'time'],
                           cmap='fire_r', title='Earthquakes with magnitude >= 7')

Now that would be a lot better with a map underneath it

In [ ]:
import pandas as pd
import datashader.geo

from holoviews.element.tiles import EsriImagery, Wikipedia, OSM

In [ ]:
x, y = datashader.geo.lnglat_to_meters(most_severe.longitude, most_severe.latitude)
most_severe_projected = most_severe.join([pd.DataFrame({'x': x}), pd.DataFrame({'y': y})])

In [ ]:
OSM() * most_severe_projected.hvplot.scatter(x='x', y='y', c='mag', hover_cols=['place', 'time'], 
                                             cmap='fire_r', title='Earthquakes with magnitude >= 7')

## Overlay with another dataset
**Note:** that we'll need rasterio if we want to do this section. But it is available on defaults, so doesn't require conda-forge.

In [ ]:
import xarray as xr
import hvplot.xarray

In [ ]:
ds = xr.open_rasterio('../data/gpw_v4_population_density_rev11_2010_2pt5_min.tif')

In [ ]:
cleaned_ds = ds.where(ds.values != ds.nodatavals[0]).sel(band=1)
cleaned_ds.name = 'population'

In [ ]:
cleaned_ds

The `xarray.plot()` API is fine for plotting small sections of this dataset, but it doesn't do well with the full resolution. For that it is better to use `hvplot` to take advantage of datashading. Here we are plotting Indonesia.

In [ ]:
ROI = cleaned_ds.sel(y=slice(10, -10), x=slice(90, 110))
ROI.plot()

In [ ]:
cleaned_ds.hvplot.image(cmap='viridis', rasterize=True, height=600, width=1000) *\
most_severe.hvplot.scatter(x='longitude', y='latitude', c='mag', hover_cols=['place', 'time'], cmap='fire_r')